#Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout
from keras.callbacks import TensorBoard
from keras.layers import Dense, Flatten, Dropout
from keras.models import Model
num_classes = 10
epochs = 300

#2 Getting the Dataset ready

2.1 Read the Dataset

In [3]:
import os

# List the contents of the directory
directory = '/kaggle/input/wiki-crop/wiki_crop/train'
print(os.listdir(directory))

['wave-offset_backdoored', 'fixed_mask_backdoored', 'gaussian_noise_backdoored', 'integral_backdoored', 'Non Backdoored', 'differential_backdoored']


In [4]:
from PIL import Image
import os

# Define the directory path
directory_path = '/kaggle/input/wiki-crop/wiki_crop/train'

# List all files in the directory
file_names = os.listdir(directory_path)

# Load images from the directory
images = []
for file_name in file_names:
    if file_name.endswith('.png') or file_name.endswith('.jpg') or file_name.endswith('.jpeg'):
        image_path = os.path.join(directory_path, file_name)
        image = Image.open(image_path)
        images.append(image)

# Process the images as required
# ...

# Example: Showing the first image
if images:
    images[0].show()
else:
    print("No images found in the directory.")

No images found in the directory.


# Define the training parameters

In [9]:
batch_size = 128
num_epochs = 300
image_size = (64, 64)
num_classes = 6

# Load the Densenet121 model

In [10]:
# Load the DenseNet121 model
image_input = Input(shape=(64, 64, 3))  # Adjust the shape according to the model's expected input shape
densenet121 = DenseNet121(include_top=False, weights='imagenet', input_tensor=image_input)

# Add a new classification layer

In [11]:
# Add a new classification layer
x = densenet121.output
x = Flatten()(x)  # Ensure that the output shape here is defined
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
class_outputs = Dense(6, activation='softmax')(x)

# Create the model
model = Model(inputs=densenet121.input, outputs=class_outputs)


#Creating,training and testing the model

In [12]:
# Create the model
model = Model(image_input, class_outputs)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

# Load the training data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
)

train_dataset = train_datagen.flow_from_directory(
    '/kaggle/input/wiki-crop/wiki_crop/train',
    target_size=(64, 64),
    batch_size=batch_size,
    class_mode='categorical'
)

# Load the validation data
val_datagen = ImageDataGenerator(rescale=1./255)

val_dataset = val_datagen.flow_from_directory(
    '/kaggle/input/wiki-crop/wiki_crop/val',
    target_size=(64, 64),
    batch_size=batch_size,
    class_mode='categorical'
)

# Train the model
model.fit(train_dataset, epochs=num_epochs, validation_data=val_dataset)

# Evaluate the model on the test data
test_datagen = ImageDataGenerator(rescale=1./255)

test_dataset = test_datagen.flow_from_directory(
    '/kaggle/input/wiki-crop/wiki_crop/test',
    target_size=(64, 64),
    batch_size=batch_size,
    class_mode='categorical'
)


Found 38030 images belonging to 6 classes.
Found 5079 images belonging to 6 classes.
Epoch 1/300
298/298 [==============================] - 272s 674ms/step - loss: 1.1468 - accuracy: 0.4125 - val_loss: 1.8205 - val_accuracy: 0.3908
Epoch 2/300
298/298 [==============================] - 107s 360ms/step - loss: 1.0232 - accuracy: 0.4412 - val_loss: 1.0309 - val_accuracy: 0.4416
Epoch 3/300
298/298 [==============================] - 114s 381ms/step - loss: 0.9973 - accuracy: 0.4509 - val_loss: 1.8020 - val_accuracy: 0.4564
Epoch 4/300
298/298 [==============================] - 116s 389ms/step - loss: 1.0050 - accuracy: 0.4542 - val_loss: 6.2344 - val_accuracy: 0.4030
Epoch 5/300
298/298 [==============================] - 113s 379ms/step - loss: 0.9657 - accuracy: 0.4840 - val_loss: 1.6067 - val_accuracy: 0.4540
Epoch 6/300
298/298 [==============================] - 110s 368ms/step - loss: 0.8084 - accuracy: 0.5738 - val_loss: 2.5913 - val_accuracy: 0.4741
Epoch 7/300
298/298 [============

#Final testing accuracy

In [13]:
from tensorflow.keras.utils import to_categorical
import numpy as np
from sklearn.metrics import classification_report
import time

test_labels = test_dataset.classes
test_labels = to_categorical(test_labels, num_classes=6)

start_time = time.time()
y_pred = model.predict(test_dataset)
y_pred_bool = np.argmax(y_pred, axis=1)
rounded_labels=np.argmax(test_labels, axis=1)
from sklearn.metrics import classification_report
print(classification_report(y_pred_bool, rounded_labels, digits=4))
print("Time taken to predict the model " + str(time.time() - start_time))

79/79 [==============================] - 62s 768ms/step
              precision    recall  f1-score   support

           0     0.2829    0.1712    0.2133      2710
           1     0.0006    0.0476    0.0011        21
           2     0.6811    0.1990    0.3080      6901
           3     0.0085    0.1161    0.0159       112
           4     0.0007    0.0303    0.0013        33
           5     0.0273    0.1520    0.0463       296

    accuracy                         0.1883     10073
   macro avg     0.1668    0.1194    0.0977     10073
weighted avg     0.5436    0.1883    0.2699     10073

Time taken to predict the model 62.782782554626465
